In [1]:
#importei dotenv para puxar a api a partir de um arquivo local

from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
import os

from groq import Groq

#Aqui estou fazendo uma chamando a API

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
#chat completion inicia um chat que nesse caso está recebendo uma mensagem pré definida a título de exemplo.
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's technology landscape, and their importance can be understood from several perspectives:

1. **Efficient Processing**: Fast language models can process and analyze large amounts of text data quickly, which is essential for applications that require real-time or near-real-time processing, such as chatbots, virtual assistants, and language translation software.
2. **Improved User Experience**: Fast language models enable applications to respond quickly to user input, providing a seamless and interactive experience. This is particularly important for applications that require immediate feedback, such as language translation, sentiment analysis, and text summarization.
3. **Scalability**: Fast language models can handle a large volume of requests and data, making them ideal for large-scale applications, such as social media platforms, search engines, and content management systems.
4. **Cost-Effectiveness**: Fast language models can reduce the com

In [61]:
#aqui criamos a classe Agent, que recebe client e o prompt do sistema system como entrada, além de criar o self.messages, que será o histórico do nosso chat
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None:
            self.messages.append({"role": "system", "content": self.system})
#call permite que a instância da classe Agent seja usada como um função, recebendo a mensagem do usuário como entrada para gerar um resultado result
    def __call__(self, message=""):
        if message: 
            self.messages.append({"role":"user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
#função que irá ser executado no __call__, ela inicia o modelo de linguagem    
    def execute(self):
        completion = client.chat.completions.create(
            messages = self.messages,
            model = "llama3-70b-8192"
        )
        return completion.choices[0].message.content

In [62]:
#prompt do sistema
system_prompt = """

You run in a loop of Thought, Action, Pause, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you habe been asked.
Use Action to run one of the action available to you - then return PAUSE.
Observation will be the result of running those actions.

You available actions are:

calculat:
e.g. calculate: 4 * 7 / 3
Runs a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

get_planet_relative_position:
e.g. get_planet_relative_position: Earth
returns Earths ordinal position as a string

Example session:

Question: What is the mass of Earth times 2 and what is the relative ordinal position?
Thought: I neet to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this: 

Observation: 5.972e24

Thought: I need to multiply this by 2 
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 5.972e24

Thoutht: I need to multiply this by 2 
Action: calculate  5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1.1944x10e25

Thoutht: now I need to find what is the relative position
Action: get_planet_relative_location: earth
PAUSE


If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,944x10e25 and your relative position is third. 

Now its your turn:

""".strip()

In [ ]:
# Ferramentas (Tools). No vídeo original há apenas 2, eu adicionei a ferramenta para descobrir a posição do planeta em relação ao sol
def calculate(operation):
    return eval(operation)

def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case _:
            return 0.0

def get_planet_relative_position(planet):
    match planet.lower():
        case "earth":
            return "third"
        case "mercury":
            return "first"
        case "venus":
            return "second"
        case "mars":
            return "fourth"
        case "jupiter":
            return "fifth"
        case "saturn":
            return "sixty"
        case _:
            return None

    

In [ ]:
#instância da classe
neil_tyson = Agent(client, system_prompt)

In [ ]:
#primeiro resultado
result = neil_tyson("what is the mass of mercury and what is your relative position to solar star?")
print(result)

Thought: I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE


In [ ]:
#histórico de mensagens
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nget_planet_relative_position:\ne.g. get_planet_relative_position: Earth\nreturns Earths ordinal position as a string\n\nExample session:\n\nQuestion: What is the mass of Earth times 2 and what is the relative ordinal position?\nThought: I neet to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThou

In [ ]:
#observação que recebe a saída da funçao
observation01 = get_planet_mass("Mercury")
print(observation01)

3.285e+23


In [ ]:
#próximo prompt que receberá a observação como entrada
next_prompt = f"Observation: {observation01}"
result = neil_tyson(next_prompt)
print(result)

Thought: Now I need to find the relative position of Mercury to the solar star.

Action: get_planet_relative_position: Mercury
PAUSE


In [ ]:
#histórico de mensagens
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nget_planet_relative_position:\ne.g. get_planet_relative_position: Earth\nreturns Earths ordinal position as a string\n\nExample session:\n\nQuestion: What is the mass of Earth times 2 and what is the relative ordinal position?\nThought: I neet to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThou

In [ ]:
#observação que recebe a saída da funçao
observation02 = get_planet_relative_position("Mercury")
print(observation02)

first


In [ ]:
#próximo prompt que receberá a observação como entrada e terá como saída o resultado esperado
next_prompt = f"Observation:{observation02}"
result = neil_tyson(next_prompt02)
print(result)

Thought: I have all the information I need.

Answer: The mass of Mercury is 3.285e+23 kg and its relative position to the solar star is first.


In [ ]:
#histórico de mensagens
neil_tyson.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, Pause, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you habe been asked.\nUse Action to run one of the action available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYou available actions are:\n\ncalculat:\ne.g. calculate: 4 * 7 / 3\nRuns a calculatation and returns the number - uses python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nget_planet_relative_position:\ne.g. get_planet_relative_position: Earth\nreturns Earths ordinal position as a string\n\nExample session:\n\nQuestion: What is the mass of Earth times 2 and what is the relative ordinal position?\nThought: I neet to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this: \n\nObservation: 5.972e24\n\nThou

In [ ]:
import re
# Essa classe representa tudo que foi feito anteriormente de forma manual, dessa vez de forma automática.
#A função recebe o prompt do sistema, o número máximo de iterções do loop e o prompt do usuário.
def agent_loop(max_iterations, system, query):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "get_planet_mass", "get_planet_relative_position"]
    next_prompt = query
    i = 0
    #O loop será executado até que i seja menor que o número de iterações definido
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)
#Nessa condicional é consultado se as strings necessárias estão presentes nos resultados e caso estejam, a função re.findall acessa o valor da ação nos resultados e o seu argumento
        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
#A partir do valor encontrado, alguma das ferramentas é executada com o argumento encontrado
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"

            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break
#Após a i-ésima iteração o loop é finalizado e temos o resultado da query presente na variável result


In [104]:
agent_loop(10, system = system_prompt, query="what is the mass of Mercury, and All what is the relative position")

Thought: I need to find the mass of Mercury.
Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I have the mass of Mercury, now I need to find its relative position.
Action: get_planet_relative_position: Mercury
PAUSE
Observation: first
Answer: The mass of Mercury is 3.285e+23 kg and its relative position is first.
